In [1]:
import torch
import pandas as pd
from glob import glob
from shutil import copyfile
from tqdm import tqdm
from torch import nn
from pytorchvideo.models.head import create_res_basic_head

In [51]:
src = 'annotations/standard/json/obfu/all/'
target = 'annotations/standard/json/obfu/test/'
all_annotations = glob(f"{src}**/*.json", recursive=True)

In [52]:
train = pd.read_csv('splits/testdata.txt', names=['videos']).values.tolist()
for i in tqdm(train):
    for j in all_annotations:
        if next(iter(i)) == j.split('/')[-1].split('.')[0]:
            copyfile(j, f"{target}/{i[0]}.json")

100%|██████████| 75/75 [00:00<00:00, 1013.72it/s]


In [8]:
model = torch.hub.load(
    "facebookresearch/pytorchvideo:main", model='i3d_r50', pretrained=True
)
backbone = nn.Sequential(*list(model.children())[0][:-1])
head = create_res_basic_head(in_features=2048, out_features=128)
model.fc = nn.Linear(in_features=128, out_features=9)

Using cache found in /home/dl18206/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [9]:
head(backbone(torch.rand(1, 3, 16, 224, 224))).shape

torch.Size([1, 128])

In [6]:
model

Net(
  (blocks): ModuleList(
    (0): ResNetBasicStem(
      (conv): Conv3d(3, 64, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
      (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
    (1): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (branch1_norm): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(64, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
            (norm_a): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(64, 64, kernel_size=(1,